In [1]:
import numpy as np
import pandas as pd
import pymysql

In [2]:
# Unpack locally stored database info

info = open('dbinfo.txt')
host, user, psswd, db = [i.split(' ')[1].replace('"', '').replace('\n', '').rstrip() for i in info]

In [3]:
db = pymysql.connect(host= host, user= user, passwd= psswd, db= db)
c = db.cursor()

In [4]:
# c.execute("SELECT * from demo.HumanEvaluationsABComparison WHERE prompt_id<201") # for the ncm
# c.execute("SELECT * from demo.HumanEvaluationsABComparison WHERE 200 < prompt_id<401") # DBDC
# c.execute("SELECT * from demo.HumanEvaluationsABComparison WHERE 2600 < prompt_id<2801") # Twitter
# c.execute("SELECT * from demo.HumanEvaluationsABComparison WHERE 3000 < prompt_id<4001") # Cornell
# c.execute("SELECT * from demo.HumanEvaluationsABComparison WHERE 4000 < prompt_id<4200") # ESL_3turn
c.execute("SELECT * from demo.HumanEvaluationsABComparison WHERE 4000 < prompt_id<4200") # Movie
raw_eval_data = c.fetchall()
c.execute("SELECT * from demo.HumanEvaluations")
mturk_runs = c.fetchall()
c.execute('SELECT model_id, name from demo.Model')
model_names = c.fetchall()
c.execute('SELECT prompt_id, prompt_text from demo.EvaluationDatasetText')
prompt_text_list = c.fetchall()
# c.execute('SELECT model_id, prompt_id, response_text from demo.ModelResponse')
# model_responses = c.fetchall()

In [5]:
models = dict(model_names)
promptid2text = dict(prompt_text_list)
model_comparisons = {}
for (mturk_run_id, _, _, _, m1, m2) in mturk_runs:
    model_comparisons[mturk_run_id] = (models[m1].replace(' ', '_') + '-' + models[m2].replace(' ','_'),m1,m2)
    # print(model_comparisons)
model_text = {}

# for (model_id, prompt_id, response_text) in model_responses: # hack for the ESL_3turn
#     if model_id not in model_text:
#         model_text[model_id] = {}
#     model_text[model_id][prompt_id] = response_text
data = []

for (comp_id, worker, hit, dt, vote, mturk_run_id, prompt_id) in raw_eval_data:
    m1 = model_comparisons[mturk_run_id][1]
    m2 = model_comparisons[mturk_run_id][2]
    # data.append((worker, mturk_run_id, prompt_id, m1, m2, promptid2text[prompt_id], 
    #              model_text[m1][prompt_id], model_text[m2][prompt_id], vote, dt))
    data.append((worker, mturk_run_id, prompt_id, m1, m2, promptid2text[prompt_id], vote, dt)) # hack for the ESL_3turn

In [6]:
# datadf = pd.DataFrame(data, columns=['worker', 'mturk_run_id', 'prompt_id', 'model 1', 'model 2', 'prompt_text', 'model 1 resp', 'model 2 resp', 'vote', 'date'])
datadf = pd.DataFrame(data, columns=['worker', 'mturk_run_id', 'prompt_id', 'model 1', 'model 2', 'prompt_text', 'vote', 'date']) # hack for the ESL_3turn
datadf

,worker,mturk_run_id,prompt_id,model 1,model 2,prompt_text,vote,date
0,A1QMM9XO2C16R1,1,1,31,5,what is two plus two ?,1,2018-03-15 04:06:25
1,A179BV6UDL0QD4,1,1,31,5,what is two plus two ?,0,2018-03-15 02:45:41
2,A1QMM9XO2C16R1,1,1,31,5,what is two plus two ?,0,2018-03-15 04:02:04
3,A2142IL8LQ5RL5,1,2,31,5,what is your name ?,-1,2018-03-15 03:58:34
4,ABCMQPOVOV0PQ,1,2,31,5,what is your name ?,-1,2018-03-15 03:12:08
...,...,...,...,...,...,...,...,...
99291,A345TDMHP3DQ3G,126,4199,112,74,A Can you please shut up?\nB Maybe if you ask...,0,2020-07-18 04:24:23
99292,A2FNAYH5134XJ4,126,4199,112,74,A Can you please shut up?\nB Maybe if you ask...,0,2020-07-18 04:24:23
99293,A307GIMK5CZLVY,126,4200,112,74,A What are those wipes for?\nB You use them t...,-1,2020-07-18 04:24:23
99294,A2WNW8A4MOR7T7,126,4200,112,74,A What are those wipes for?\nB You use them t...,0,2020-07-18 04:24:23


In [7]:
new = datadf.loc[(datadf['model 1'] == 100) & (datadf['model 2'] == 113)] 
# datadf
# datadf.to_excel('turker.xlsx')
new

,worker,mturk_run_id,prompt_id,model 1,model 2,prompt_text,vote,date
88502,A1DMP3A9A68VSI,109,4001,100,113,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-18 00:21:32
88503,A2E2ZE0SOY9UPW,109,4001,100,113,"A It's been very sunny lately, hasn't it?\nB ...",1,2020-07-18 00:21:32
88504,A182N7RLXGSCZG,109,4001,100,113,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-18 00:21:32
88505,A2179410P3XBIQ,109,4002,100,113,A What are you going to do about your death?\n...,0,2020-07-18 00:21:32
88506,A1B7SDXORQ2QBX,109,4002,100,113,A What are you going to do about your death?\n...,1,2020-07-18 00:21:32
...,...,...,...,...,...,...,...,...
89097,AFNP1C7IWHEPZ,109,4199,100,113,A Can you please shut up?\nB Maybe if you ask...,-1,2020-07-18 00:21:32
89098,A1L1S0IAPZB4MO,109,4199,100,113,A Can you please shut up?\nB Maybe if you ask...,1,2020-07-18 00:21:32
89099,A3CNED54M9CQTJ,109,4200,100,113,A What are those wipes for?\nB You use them t...,0,2020-07-18 00:21:32
89100,A2E2ZE0SOY9UPW,109,4200,100,113,A What are those wipes for?\nB You use them t...,0,2020-07-18 00:21:32


In [8]:
pd.DataFrame(model_names, columns=['model id', 'model name'])

,model id,model name
0,5,CakeChat
1,83,Chatbot
2,84,Chatbot with new embeddings
3,86,ChatbotII
4,20,Cornell Movie DC Baseline
...,...,...
58,22,Transformer
59,21,Transformer (target filtered)
60,23,Transformer (target filtered) (not overfit)
61,19,Twitter Baseline


In [9]:
new = datadf.loc[(datadf['model 1'] == 112) & (datadf['model 2'] == 100)] 

category = [-1, 0, 1]
# A/B comparison  0 : A win / 1: B win / -1: tie 
one_person_disagree_count = 0
all_agreement_count = 0
all_disagreement_count = 0
a_b_disagreement_count = 0
  

temp_list = []

for data in range(4001, 4201): #movie
# for data in range(4001, 4201): #ESL_3turn
# for data in range(3001, 4001): #Cornell
# for data in range(2601, 2801): #Twitter
# for data in range(201, 401): #DBDC
# for data in range(1, 201): #NCM
  label_count = 0
  voter_person_count = len(new.loc[(new['prompt_id'] == data)])
  if voter_person_count == 3: # three voters
    label_1 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[0]) ]) # tie
    label_2 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[1]) ]) # A win
    label_3 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[2]) ]) # B win

    temp_list = [label_1, label_2, label_3] #[2, 0, 1] means 2 voters pick the tie and the other one pick B win
    # print(temp_list)
  
    if temp_list == [0,2,1] or temp_list == [0,1,2]: #I was intending that disagreement was when I chose A and you chose B, But if I chose tie then  it isn't a disagreement
      a_b_disagreement_count += 1

    if 2 in temp_list:
      one_person_disagree_count += 1
    if 3 in temp_list:
      all_agreement_count += 1
    if temp_list[0] == 1 and temp_list[1] == 1 and temp_list[2] == 1:
      all_disagreement_count += 1
      
  # elif voter_person_count == 2: # only 2 voters
  #   print("Found!!")
  #   label_1 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[0]) ])
  #   label_2 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[1]) ])
  #   label_3 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[2]) ])

  #   temp_list = [label_1, label_2, label_3]

  #   if temp_list == [0,1,1] or temp_list == [2,0,0]: #only cares A/B who wins (not consider the tie)
  #     a_b_disagreement_count += 1

  #   if 1 in temp_list:
  #     all_disagreement_count += 1
  #   elif 2 in temp_list:
  #     all_agreement_count += 1
  
  elif voter_person_count == 5: # five voters
    label_1 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[0]) ])
    label_2 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[1]) ])
    label_3 = len(new.loc[(new['prompt_id'] == data) & (new['vote'] == category[2]) ])

    temp_list = [label_1, label_2, label_3]

    if temp_list == [0, 2, 3] or temp_list == [0,3,2] or temp_list == [0,1,4]  or temp_list == [0,4,1]: #only cares A/B who wins (not consider the tie)
      a_b_disagreement_count += 1

    if 2 in temp_list:
      one_person_disagree_count += 1
    elif 3 in temp_list:
      one_person_disagree_count += 1
    elif 4 in temp_list:
      one_person_disagree_count += 1
    if 5 in temp_list:
      all_agreement_count += 1
    # if temp_list[0] == 1 and temp_list[1] == 1 and temp_list[2] == 1:
    #   all_disagreement_count += 1
    


print("All agreed:", all_agreement_count)
print("A/B disagreed:", a_b_disagreement_count )
print("one person diagreement:", one_person_disagree_count)
print("All disagreed:", all_disagreement_count )

total = one_person_disagree_count + all_agreement_count + all_disagreement_count 
print("Total voting counts by utterance (200):", total)

All agreed: 36
A/B disagreed: 21
one person diagreement: 122
All disagreed: 42
Total voting counts by utterance (200): 200


In [10]:
#ncm mturk_run_id: 5,6, 19 - 60, 101
# datadf_new_1 = datadf.loc[(datadf['mturk_run_id'] >= 5) & (datadf['mturk_run_id'] <= 6)]
# datadf_new_2 = datadf.loc[(datadf['mturk_run_id'] >= 19) & (datadf['mturk_run_id'] <= 60)] 
# datadf_new_3 = datadf.loc[(datadf['mturk_run_id'] == 101)] 
# frames = [datadf_new_1, datadf_new_2, datadf_new_3]
# datadf_new = pd.concat(frames)
# datadf_new

#dbdc mturk_run_id: 61 - 75 
# datadf_new = datadf.loc[(datadf['mturk_run_id'] >= 61) & (datadf['mturk_run_id'] <= 75)]
# datadf_new

#twitter mturk_run_id: 76 - 90 
# datadf_new = datadf.loc[(datadf['mturk_run_id'] >= 76) & (datadf['mturk_run_id'] <= 90)]
# datadf_new

#cornell mturk_run_id: 91 - 100 
# datadf_new = datadf.loc[(datadf['mturk_run_id'] >= 91) & (datadf['mturk_run_id'] <= 100)]
# datadf_new

#ESL_3turn mturk_run_id: 105 - 126
datadf_new = datadf.loc[(datadf['mturk_run_id'] >= 105) & (datadf['mturk_run_id'] <= 126)]
datadf_new

,worker,mturk_run_id,prompt_id,model 1,model 2,prompt_text,vote,date
86114,A17DZTQCQMPC9A,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40
86115,A1TK9VM3WZDAGP,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40
86116,A421VR3CYMQKN,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40
86117,A345TDMHP3DQ3G,105,4002,100,50,A What are you going to do about your death?\n...,0,2020-07-17 23:32:40
86118,A1L1S0IAPZB4MO,105,4002,100,50,A What are you going to do about your death?\n...,0,2020-07-17 23:32:40
...,...,...,...,...,...,...,...,...
99291,A345TDMHP3DQ3G,126,4199,112,74,A Can you please shut up?\nB Maybe if you ask...,0,2020-07-18 04:24:23
99292,A2FNAYH5134XJ4,126,4199,112,74,A Can you please shut up?\nB Maybe if you ask...,0,2020-07-18 04:24:23
99293,A307GIMK5CZLVY,126,4200,112,74,A What are those wipes for?\nB You use them t...,-1,2020-07-18 04:24:23
99294,A2WNW8A4MOR7T7,126,4200,112,74,A What are those wipes for?\nB You use them t...,0,2020-07-18 04:24:23


In [12]:
# column info: 
# scrlan,trglan == evalset name
# srcindex, segmentid == prompt_id
# judgeId == worker_id
# system1Id == Blender_id
# system2Id == DialoGPT_id 
# system1rank == 1 if blender wins 
# system2rank == 2 if blender wins

ranking = datadf_new
ranking['system1rank'] = 0
ranking['system2rank'] = 0
ranking['srclang'] = 'ESL3turn'
ranking['trglang'] = 'ESL3turn'
ranking['srclndex'] = 0
ranking['documentId'] = -1
ranking['system1Number'] = -1
ranking['segmentId'] = ranking['prompt_id']
ranking['system2Number'] = -1

for index, row in ranking.iterrows():
  if row['vote'] == 0:
    row['system1rank'] = ranking.loc[index, 'system1rank'] = 1
    row['system2rank'] = ranking.loc[index, 'system2rank'] = 2
  elif row['vote'] == 1:
    row['system1rank'] = ranking.loc[index, 'system1rank'] = 2
    row['system2rank'] = ranking.loc[index, 'system2rank'] = 1
  elif row['vote'] == -1:
    row['system1rank'] = ranking.loc[index, 'system1rank'] = 1
    row['system2rank'] = ranking.loc[index, 'system2rank'] = 1

ranking = ranking.rename(columns={"model 1": "system1Id", "model 2": "system2Id", "prompt_id": "srcIndex", "worker": "judgeId"})
columnsTitles = ['mturk_run_id','srclang', 'trglang', 'srcIndex', 'documentId', 'segmentId', 'judgeId', 'system1Number', 'system1Id', 'system2Number', 'system2Id', 'system1rank', 'system2rank']
frame = ranking.reindex(columns=columnsTitles)

ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:14: SettingWithCopyWarning: 
A value is trying to b

In [13]:
frame

,mturk_run_id,srclang,trglang,srcIndex,documentId,segmentId,judgeId,system1Number,system1Id,system2Number,system2Id,system1rank,system2rank
86114,105,ESL3turn,ESL3turn,4001,-1,4001,A17DZTQCQMPC9A,-1,100,-1,50,1,2
86115,105,ESL3turn,ESL3turn,4001,-1,4001,A1TK9VM3WZDAGP,-1,100,-1,50,1,2
86116,105,ESL3turn,ESL3turn,4001,-1,4001,A421VR3CYMQKN,-1,100,-1,50,1,2
86117,105,ESL3turn,ESL3turn,4002,-1,4002,A345TDMHP3DQ3G,-1,100,-1,50,1,2
86118,105,ESL3turn,ESL3turn,4002,-1,4002,A1L1S0IAPZB4MO,-1,100,-1,50,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99291,126,ESL3turn,ESL3turn,4199,-1,4199,A345TDMHP3DQ3G,-1,112,-1,74,1,2
99292,126,ESL3turn,ESL3turn,4199,-1,4199,A2FNAYH5134XJ4,-1,112,-1,74,1,2
99293,126,ESL3turn,ESL3turn,4200,-1,4200,A307GIMK5CZLVY,-1,112,-1,74,1,1
99294,126,ESL3turn,ESL3turn,4200,-1,4200,A2WNW8A4MOR7T7,-1,112,-1,74,1,2


In [14]:
#ncm mturk_run_id: 5,6, 19 - 60, 101
# datadf_new_1 = datadf.loc[(datadf['mturk_run_id'] >= 5) & (datadf['mturk_run_id'] <= 6)]
# datadf_new_2 = datadf.loc[(datadf['mturk_run_id'] >= 19) & (datadf['mturk_run_id'] <= 60)] 
# datadf_new_3 = datadf.loc[(datadf['mturk_run_id'] == 101)] 
# frames = [datadf_new_1, datadf_new_2, datadf_new_3]
# datadf_new = pd.concat(frames)
# datadf_new

#dbdc mturk_run_id: 61 - 75 
# datadf_new = datadf.loc[(datadf['mturk_run_id'] >= 61) & (datadf['mturk_run_id'] <= 75)]
# datadf_new

#twitter mturk_run_id: 76 - 90 
# datadf_new = datadf.loc[(datadf['mturk_run_id'] >= 76) & (datadf['mturk_run_id'] <= 90)]
# datadf_new

#cornell mturk_run_id: 91 - 100 
# datadf_new = datadf.loc[(datadf['mturk_run_id'] >= 91) & (datadf['mturk_run_id'] <= 100)]
# datadf_new

#ESL_3turn mturk_run_id: 105 - 126
datadf_new = datadf.loc[(datadf['mturk_run_id'] >= 105) & (datadf['mturk_run_id'] <= 126)]
datadf_new

,worker,mturk_run_id,prompt_id,model 1,model 2,prompt_text,vote,date
86114,A17DZTQCQMPC9A,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40
86115,A1TK9VM3WZDAGP,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40
86116,A421VR3CYMQKN,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40
86117,A345TDMHP3DQ3G,105,4002,100,50,A What are you going to do about your death?\n...,0,2020-07-17 23:32:40
86118,A1L1S0IAPZB4MO,105,4002,100,50,A What are you going to do about your death?\n...,0,2020-07-17 23:32:40
...,...,...,...,...,...,...,...,...
99291,A345TDMHP3DQ3G,126,4199,112,74,A Can you please shut up?\nB Maybe if you ask...,0,2020-07-18 04:24:23
99292,A2FNAYH5134XJ4,126,4199,112,74,A Can you please shut up?\nB Maybe if you ask...,0,2020-07-18 04:24:23
99293,A307GIMK5CZLVY,126,4200,112,74,A What are those wipes for?\nB You use them t...,-1,2020-07-18 04:24:23
99294,A2WNW8A4MOR7T7,126,4200,112,74,A What are those wipes for?\nB You use them t...,0,2020-07-18 04:24:23


In [15]:
# column info:
# date, Player A, Player B, Wins A, Wins B 
# scrlan,trglan == evalset name
# srcindex, segmentid == prompt_id
# judgeId == worker_id
# system1Id == Blender_id
# system2Id == DialoGPT_id 
# system1rank == 1 if blender wins 
# system2rank == 2 if blender wins

ranking = datadf_new
ranking['Wins A'] = 0
ranking['Wins B'] = 0


for index, row in ranking.iterrows():
  ranking['date'] = pd.to_datetime(ranking.loc[index, 'date']) 
  if row['vote'] == 0:
    row['Wins A'] = ranking.loc[index, 'Wins A'] = 1
    row['Wins B'] = ranking.loc[index, 'Wins B'] = 0
  elif row['vote'] == 1:
    row['Wins A'] = ranking.loc[index, 'Wins A'] = 0
    row['Wins B'] = ranking.loc[index, 'Wins B'] = 1
  elif row['vote'] == -1:
    row['Wins A'] = ranking.loc[index, 'Wins A'] = 1
    row['Wins B'] = ranking.loc[index, 'Wins B'] = 1
    

ranking = ranking.rename(columns={"model 1": "Player A", "model 2": "Player B", 'date': 'Date'})
columnsTitles = ['Date', 'Player A', 'Player B', 'Wins A', 'Wins B']
frame = ranking.reindex(columns=columnsTitles)

ipykernel_launcher:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/nicksubic/opt/anaconda3/envs/learn-env/lib/python3.6/site-packa

In [16]:
ranking

,worker,mturk_run_id,prompt_id,Player A,Player B,prompt_text,vote,Date,Wins A,Wins B
86114,A17DZTQCQMPC9A,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40,1,0
86115,A1TK9VM3WZDAGP,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40,1,0
86116,A421VR3CYMQKN,105,4001,100,50,"A It's been very sunny lately, hasn't it?\nB ...",0,2020-07-17 23:32:40,1,0
86117,A345TDMHP3DQ3G,105,4002,100,50,A What are you going to do about your death?\n...,0,2020-07-17 23:32:40,1,0
86118,A1L1S0IAPZB4MO,105,4002,100,50,A What are you going to do about your death?\n...,0,2020-07-17 23:32:40,1,0
...,...,...,...,...,...,...,...,...,...,...
99291,A345TDMHP3DQ3G,126,4199,112,74,A Can you please shut up?\nB Maybe if you ask...,0,2020-07-17 23:32:40,1,0
99292,A2FNAYH5134XJ4,126,4199,112,74,A Can you please shut up?\nB Maybe if you ask...,0,2020-07-17 23:32:40,1,0
99293,A307GIMK5CZLVY,126,4200,112,74,A What are those wipes for?\nB You use them t...,-1,2020-07-17 23:32:40,1,1
99294,A2WNW8A4MOR7T7,126,4200,112,74,A What are those wipes for?\nB You use them t...,0,2020-07-17 23:32:40,1,0


In [26]:
# for r code bradley-terry running
new_brad = frame[['Player A', 'Player B', 'Wins A', 'Wins B']]

# rename column name as Team, Opponent, Wins, Losses
# data ex)
# Team Opponent Wins Losses
# BAL BOS 11 8
# BAL CLE 3 4 
# # For each team in the AL table, identifies the team's name, the opponent,    # and their wins and losses (WL) against that opponent.
new_brad.columns = ['Team', 'Opponent', 'Wins', 'Losses']
new_brad

,Team,Opponent,Wins,Losses
86114,100,50,1,0
86115,100,50,1,0
86116,100,50,1,0
86117,100,50,1,0
86118,100,50,1,0
...,...,...,...,...
99291,112,74,1,0
99292,112,74,1,0
99293,112,74,1,1
99294,112,74,1,0


In [54]:
new_brad.groupby(['Team'])['Wins'].value_counts()

Team  Wins
5     1       1190
      0        610
50    1       1881
      0        516
51    1        977
      0        220
100   1       3176
      0        412
112   1       3014
      0        586
113   1        410
      0        190
Name: Wins, dtype: int64

In [18]:
import sys

sys.path.insert(1, "../Documents/wmt-trueskill/src")
sys.path.insert(1, "../Documents/wmt-trueskill/src/scripts")
sys.path.extend(['/content/wmt-trueskill', '/content/wmt-trueskill/src'])

In [19]:
import os
# os.chdir('/content')
os.chdir('../Documents/wmt-trueskill/src')

In [20]:
!cat /content/twitter_ranking.csv |python infer_TS.py ../results/twitter_ranking -n 2 -d 0 -s 2

cat: /content/twitter_ranking.csv: No such file or directory
<_io.TextIOWrapper name='<stderr>' mode='w' encoding='utf-8'> Sampling 1 / 1 pairwise judgments
Traceback (most recent call last):
  File "infer_TS.py", line 223, in <module>
    estimate_by_number()
  File "infer_TS.py", line 168, in estimate_by_number
    systems_compared = scripts.next_comparison.get(get_mu_sigma(system_rating), args.freeN)
  File "/Users/nicksubic/Documents/wmt-trueskill/src/scripts/next_comparison.py", line 27, in get
    sys_a = sort_by_sigma(sys_musigma)[0][1]
IndexError: list index out of range
